<a href="https://colab.research.google.com/github/lcbjrrr/quant/blob/master/J_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Topic:** AI/ML

**Title:** Neural Network (with Flux)

**Author:** Luiz Barboza

**Date:** 20/dec/22

**Lang:** Julia

**Site:** https://quant-research.group/

**Email:** contato@quant-research.group


# Julia Installation

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.3" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.8.3 on the current Colab Runtime...
2023-01-06 14:37:19 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.8/julia-1.8.3-linux-x86_64.tar.gz [130030846/130030846] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.8

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then
jump to the 'Checking the Installation' section.


In [1]:
versioninfo()

Julia Version 1.8.3
Commit 0434deb161e (2022-11-14 20:14 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, broadwell)
  Threads: 2 on 2 virtual cores
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  LD_PRELOAD = /usr/lib/x86_64-linux-gnu/libtcmalloc.so.4
  JULIA_NUM_THREADS = 2


# Neural Networks (with Flux)

In [2]:
import Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("Lathe") 
Pkg.add("Flux") 
Pkg.add("StatsBase") 
Pkg.add("EvalMetrics") 

using CSV
using DataFrames
using Lathe
using Lathe.preprocess: TrainTestSplit
using Flux
using Flux: crossentropy, throttle
using StatsBase
using EvalMetrics

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed TableTraits ───────────────── v1.0.1
   Installed DataAPI ───────────────────── v1.14.0
   Installed PooledArrays ──────────────── v1.4.2
   Installed SentinelArrays ────────────── v1.3.16
   Installed Tables ────────────────────── v1.10.0
   Installed InlineStrings ─────────────── v1.3.2
   Installed IteratorInterfaceExtensions ─ v1.0.0
   Installed DataValueInterfaces ───────── v1.0.0
   Installed WorkerUtilities ───────────── v1.6.1
   Installed WeakRefStrings ────────────── v1.4.2
   Installed Compat ────────────────────── v4.5.0
   Installed OrderedCollections ────────── v1.4.1
   Installed FilePathsBase ─────────────── v0.9.20
   Installed CodecZlib ─────────────────── v0.7.0
   Installed TranscodingStreams ────────── v0.9.10
   Installed CSV ───────────────────────── v0.10.9
    Updating `~/.julia/environments/v1.8/Project.toml`
  [336ed68f] + CSV v0.10.9
    Updating `~/.ju

## NN - Classification

In [ ]:
;wget https://raw.githubusercontent.com/lcbjrrr/data/main/banking%20-%20train.csv


In [ ]:
;wget https://raw.githubusercontent.com/lcbjrrr/data/main/banking%20-%20tests.csv

In [ ]:
train=CSV.read("banking - train.csv", DataFrame)
first(train,5)

,salary,balance,default
,Float64,Float64,Int64
1,1000.0,100.0,0
2,950.0,95.0,0
3,800.0,85.0,0
4,1100.0,5500.0,0
5,999.0,6000.0,0


In [ ]:
test=CSV.read("banking - tests.csv", DataFrame)
first(train,5)

,salary,balance,default
,Float64,Float64,Int64
1,1000.0,100.0,0
2,950.0,95.0,0
3,800.0,85.0,0
4,1100.0,5500.0,0
5,999.0,6000.0,0


In [ ]:
X_train  = transpose(hcat([ row.salary for row in eachrow(train)],[ row.balance for row in eachrow(train)]))
y_train = transpose([row.default for row in eachrow(train)])
y_train

1×16 transpose(::Vector{Int64}) with eltype Int64:
 0  0  0  0  0  0  0  0  0  1  1  1  0  0  0  0

In [ ]:
EP = 20
N=length(y_train)
BS=floor(Int,N/EP)
data = Flux.Data.DataLoader((X_train, y_train), batchsize=BS,shuffle=true);
nn    = Chain(
  Dense(2,4, relu),
  Dense(4,3, relu),
  Dense(3,1,identity),
  softmax
  ) 
loss(x, y) = Flux.Losses.binarycrossentropy(nn(x), y)
for i in 1:EP
    # train (loss func, NN params, data/batches, optimizer)
    Flux.train!(loss, Flux.params(nn), data,ADAM())
end
pred_train  =  transpose([nn(X_train[i:i+1])[1] for i in 1:2:N*2])
pred_train

1×16 transpose(::Vector{Float64}) with eltype Float64:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0

In [ ]:
function accuracy(y,pred)
  acc = sum(pred.==y)/length(y)
  return acc
end

acc_train = accuracy(y_train, pred_train)
print("Acc. Train: ",acc_train)

Acc. Train: 0.1875

In [ ]:
X_test = transpose(hcat([ row.salary for row in eachrow(test)],[ row.balance for row in eachrow(test)]))
y_test = transpose([row.default for row in eachrow(test)])
print(y_test)


[1 0 0 0 0 0 0 1 0 1]

In [ ]:
pred_test  =  transpose([ Int64(nn(X_test[i:i+1])[1]) for i in 1:2:length(y_test)*2])
print(pred_test)

[1 1 1 1 1 1 1 1 1 1]

In [ ]:
acc_test = accuracy(y_test, pred_test)
print("Acc. Test: ",acc_test)

Acc. Test: 0.3

In [ ]:
counts( y_test,pred_test)

2×1 Matrix{Int64}:
 7
 3

In [ ]:
println("Precision: ",precision(vec(y_test),vec(pred_test)) )
println("Recall: " , recall(vec(y_test),vec(pred_test)) )

Precision: 0.3
Recall: 1.0


# Second Example: Gender

In [3]:
;wget https://raw.githubusercontent.com/lcbjrrr/data/main/gender%20-%20all.csv

--2023-01-06 14:46:59--  https://raw.githubusercontent.com/lcbjrrr/data/main/gender%20-%20all.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 267751 (261K) [text/plain]
Saving to: ‘gender - all.csv’

     0K .......... .......... .......... .......... .......... 19% 6.68M 0s
    50K .......... .......... .......... .......... .......... 38% 8.78M 0s
   100K .......... .......... .......... .......... .......... 57% 31.4M 0s
   150K .......... .......... .......... .......... .......... 76% 26.8M 0s
   200K .......... .......... .......... .......... .......... 95% 13.7M 0s
   250K .......... .                                          100%  201M=0.02s

2023-01-06 14:47:00 (12.9 MB/s) - ‘gender - all.csv’ saved [267751/267751]



In [8]:
df=CSV.read("gender - all.csv", DataFrame)
first(df,5)

,G,H,W
,Int64,Float64,Float64
1,0,187.571,109.952
2,0,174.706,73.7775
3,0,188.24,96.7004
4,0,182.197,100.019
5,0,177.5,93.7954


In [9]:
train, test = TrainTestSplit(df,.70)

(6978×3 DataFrame
  Row │ G      H        W        
      │ Int64  Float64  Float64  
──────┼──────────────────────────
    1 │     0  187.571  109.952
    2 │     0  174.706   73.7775
    3 │     0  182.197  100.019
    4 │     0  177.5     93.7954
    5 │     0  170.823   69.1873
    6 │     0  174.714   83.6036
    7 │     0  173.605   76.3505
    8 │     0  170.228   79.9679
    9 │     0  161.179   71.0908
   10 │     0  180.836   84.8204
   11 │     0  164.506   75.967
  ⋮   │   ⋮       ⋮        ⋮
 6969 │     1  154.826   43.4939
 6970 │     1  160.476   64.2119
 6971 │     1  157.678   63.0417
 6972 │     1  152.477   44.4034
 6973 │     1  150.11    50.2408
 6974 │     1  168.079   62.1716
 6975 │     1  170.351   77.6672
 6976 │     1  162.225   58.3979
 6977 │     1  175.347   74.4784
 6978 │     1  157.338   51.6587
                6957 rows omitted, 3022×3 DataFrame
  Row │ G      H        W        
      │ Int64  Float64  Float64  
──────┼──────────────────────────
    1 │

In [10]:
X_train  = transpose(hcat([ row.H for row in eachrow(train)],[ row.W for row in eachrow(train)]))
y_train = transpose([row.G for row in eachrow(train)])
y_train

1×6978 transpose(::Vector{Int64}) with eltype Int64:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  1  1  1  1  1  1  1  1  1  1  1  1

In [11]:
EP = 20
N=length(y_train)
BS=floor(Int,N/EP)
data = Flux.Data.DataLoader((X_train, y_train), batchsize=BS,shuffle=true);
nn    = Chain(
  Dense(2,4, relu),
  Dense(4,3, relu),
  Dense(3,1,identity),
  softmax
  ) 
loss(x, y) = Flux.Losses.binarycrossentropy(nn(x), y)
for i in 1:EP
    # train (loss func, NN params, data/batches, optimizer)
    Flux.train!(loss, Flux.params(nn), data,ADAM(), cb=throttle(() -> @show(loss(X_train, y_train)), 1))
end
pred_train  =  transpose([nn(X_train[i:i+1])[1] for i in 1:2:N*2])
pred_train

loss(X_train, y_train) = 18.01666136733173
loss(X_train, y_train) = 18.01666136733173
loss(X_train, y_train) = 18.01666136733173
loss(X_train, y_train) = 18.01666136733173
loss(X_train, y_train) = 18.01666136733173
loss(X_train, y_train) = 18.01666136733173
loss(X_train, y_train) = 18.01666136733173
loss(X_train, y_train) = 18.01666136733173
loss(X_train, y_train) = 18.01666136733173
loss(X_train, y_train) = 18.01666136733173
loss(X_train, y_train) = 18.01666136733173
loss(X_train, y_train) = 18.01666136733173
loss(X_train, y_train) = 18.01666136733173
loss(X_train, y_train) = 18.01666136733173
loss(X_train, y_train) = 18.01666136733173
loss(X_train, y_train) = 18.01666136733173
loss(X_train, y_train) = 18.01666136733173
loss(X_train, y_train) = 18.01666136733173
loss(X_train, y_train) = 18.01666136733173
loss(X_train, y_train) = 18.01666136733173


1×6978 transpose(::Vector{Float64}) with eltype Float64:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0

In [12]:
function accuracy(y,pred)
  acc = sum(pred.==y)/length(y)
  return acc
end

acc_train = accuracy(y_train, pred_train)
print("Acc. Train: ",acc_train)

Acc. Train: 0.5001433075379765

In [13]:
X_test = transpose(hcat([ row.H for row in eachrow(test)],[ row.W for row in eachrow(test)]))
y_test = transpose([row.G for row in eachrow(test)])

pred_test  =  transpose([ Int64(nn(X_test[i:i+1])[1]) for i in 1:2:length(y_test)*2])


1×3022 transpose(::Vector{Int64}) with eltype Int64:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1

In [14]:
acc_test = accuracy(y_test, pred_test)
print("Acc. Test: ",acc_test)

Acc. Test: 0.499669093315685

In [15]:
counts( y_test,pred_test)

2×1 Matrix{Int64}:
 1512
 1510

In [16]:
println("Precision: ",precision(vec(y_test),vec(pred_test)) )
println("Recall: " , recall(vec(y_test),vec(pred_test)) )

Precision: 0.499669093315685
Recall: 1.0




```
X_train  = [ [row.H,row.W] for row in eachrow(train)]
y_train = [ row.G for row in eachrow(train)]
batches = (Flux.batch(X_train),Flux.batch(y_train))
nn = Chain(
  Dense(2,4, relu),
  Dense(4,3, relu),
  Dense(3,1,identity),
  softmax
  )
L(x,y)=Flux.crossentropy(nn(x), y)
evalcb = () -> @show(L) 
for i in 1:11
  Flux.train!(L,Flux.params(nn),[batches],ADAM(), cb = Flux.throttle(evalcb, 1))
end
pred_train  = [ nn(row[1]) for row in eachrow(X_train)]

```





```
 
#Auxiliary functions for generating our data
function generate_real_data(n)
    x1 = rand(1,n) .- 0.5
    x2 = (x1 .* x1)*3 .+ randn(1,n)*0.1
    return vcat(x1,x2)
end
function generate_fake_data(n)
    θ  = 2*π*rand(1,n)
    r  = rand(1,n)/3
    x1 = @. r*cos(θ)
    x2 = @. r*sin(θ)+0.5
    return vcat(x1,x2)
end
# Creating our data
train_size = 5000
real = generate_real_data(train_size)
fake = generate_fake_data(train_size)
# Visualizing
scatter(real[1,1:500],real[2,1:500])
scatter!(fake[1,1:500],fake[2,1:500])
function NeuralNetwork()
    return Chain(
            Dense(2, 25,relu),
            Dense(25,1,x->σ.(x))
            )
end
# Organizing the data in batches
X    = hcat(real,fake)

Y    = vcat(ones(train_size),zeros(train_size))

data = Flux.Data.DataLoader((X, Y'), batchsize=100,shuffle=true);
# Defining our model, optimization algorithm and loss function
m    = NeuralNetwork()
opt = Descent(0.05)
loss(x, y) = sum(Flux.Losses.binarycrossentropy(m(x), y))
# Training Method 1
ps = Flux.params(m)
epochs = 20
for i in 1:epochs
    Flux.train!(loss, ps, data, opt)
end
println(mean(m(real)),mean(m(fake))) # Print model prediction
# Visualizing the model predictions
scatter(real[1,1:100],real[2,1:100],zcolor=m(real)')
scatter!(fake[1,1:100],fake[2,1:100],zcolor=m(fake)',legend=false)

```

